In [2]:
import csv
from datetime import datetime
import pandas as pd

In [ ]:
def period_assign(row):
    if row["effdate"] >= 201301 and row["effdate"] < 201801:
        val = 2
    elif row["effdate"] >= 200801 and row["effdate"] < 201301:
        val = 1
    elif row["effdate"] >= 200301 and row["effdate"] < 200801:
        val = -1
    else:
        val = 0
    return val

In [ ]:
z4types = ["S", "H", "R"]
long_moves = pd.DataFrame()

status_file = "status_allmoves.txt"
source_file = "data/all_states.csv"

print_line_every = 500
start_line = 1
process_records = 27408937
end_line = start_line + process_records

with open(status_file) as f:
    last_status_line = int(f.read())
resume_from_status = input(f"Type Yes to resume from last line processed in "
                           f"{status_file} which is: {last_status_line:,}\n")
if resume_from_status == "Yes":
    start_line = last_status_line + 1

with open(source_file, "r") as object:
    line_count = start_line
    process_start = datetime.now()
    csv_file = csv.reader(object)
    
    for row in csv_file:
        df = pd.DataFrame({"address_observation": [x + 1 for x in range(10)],
                           "pid": [row[0] for _ in range(10)],
                           "idate": [row[4] for _ in range(10)],
                           "odate": [row[5] for _ in range(10)],
                           "z4type": [row[9*x + 13] for x in range(10)],
                           "effdate": [row[9*x + 14] for x in range(10)],
                           "fips": [row[4*x + 99] for x in range(10)]})

        df = df[df.fips != ""]
        df = df[df["z4type"].isin(z4types)]

        df["effdate"] = pd.to_numeric(df["effdate"])
        df["idate"] = pd.to_numeric(df["idate"])
        df["period"] = ""
        df["period"] = df.apply(period_assign, axis=1, result_type="reduce")
        df["seentime"] = abs(df["odate"] - df["idate"])

        df.sort_values(by=["effdate"], ascending=True, inplace=True)
        df["originfips"] = df.fips.shift(1, fill_value="first record")
        df["prev_effdate"] = df.effdate.shift(1).astype(str)

        df.drop_duplicates(subset=["effdate"])
        df.dropna(axis=0, subset=["originfips"], how="any", inplace=True)
        long_moves = long_moves.append(df)

        line_count += 1
        if line_count % print_line_every == 0:
            long_moves.to_csv(path_or_buf="data/allstates.csv",
                              mode="a", header=False, index=False)
            long_moves.drop(long_moves.index, inplace=True)

            process_duration = datetime.now() - process_start
            rate = print_line_every / (process_duration.total_seconds()/60)
            estimated_completion = (end_line - line_count) / rate
            print(f"Finished: {line_count}, PID: {row[0]} at "
                  f"{datetime.now().time()} | rate: {rate:.1f} per min. | this "
                  f"batch complete: {estimated_completion:.1f} min")
            
            process_start = datetime.now()

            with open(status_file, "w") as writer:
                writer.write(str(line_count))